## [수정X] 캐글 사용을 위한 설정

In [58]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 66 Oct 31 10:45 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl


## [수정X]
### [주의] 반드시 Token 다운 로드 후 업로드


In [59]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

Kaggle API 1.5.6


## [수정X] 데이터 셋 다운로드 후 압축 풀기

In [60]:
! kaggle competitions download -c 2019-pr-midterm-musicclassification

import zipfile
import os


zip_ref = zipfile.ZipFile("/content/2019-pr-midterm-musicclassification.zip", 'r')
zip_ref.extractall("/content/input2")
zip_ref.close()

2019-pr-midterm-musicclassification.zip: Skipping, found more recently modified local copy (use --force to force download)


## [수정O] 라이브러리 로드
- 필요시 라이브러리 추가 가능

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from pandas.tools.plotting import scatter_matrix
from pandas.plotting import autocorrelation_plot

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import axes3d, Axes3D
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from itertools import product
from sklearn import svm

import warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

In [0]:
scaler = StandardScaler()

## [수정O] 학습 데이터 준비 
- 음성데이터는 scale 함수로 정규화 하라

In [0]:
# Load datasets
# DataFrame 을 이용하면 편리하다.
musicdata = pd.read_csv('/content/input2/data_train.csv')
## CSV 의 컬럼만큼 생성.
train = musicdata [['tempo', 'beats', 'chroma_stft', 'rmse',
       'spectral_centroid', 'spectral_bandwidth', 'rolloff',
       'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5',
       'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12',
       'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19',
       'mfcc20']]

train_labels = musicdata [['label']]

In [0]:
train = train.values
scaler.fit(train)
train = scaler.transform(train)
train_labels = train_labels.values

In [0]:
#라벨 생성
labels = list()

for label in train_labels :
  if label == 'blues':
    labels.append(0)
  elif label == 'classical':
    labels.append(1)
  elif label == 'country':
    labels.append(2)
  elif label == 'disco':
    labels.append(3)
  elif label == 'hiphop':
    labels.append(4)
  elif label == 'jazz':
    labels.append(5)
  elif label == 'metal':
    labels.append(6)
  elif label == 'pop':
    labels.append(7)
  elif label == 'reggae':
    labels.append(8)
  else :
    labels.append(9)

labels = np.array(labels)


## [수정O] 데이터 셋 서브 샘플링 & Train-Validation 나누기
- 아래의 코드를 참고하여 사용
- 반드시 Random Seed 42로 설정
- 25% Validation 으로 사용 권장

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.25, random_state=42)

## [수정 O] 학습 기반 분류기 설계
- 본인이 알고 있는 방법 중 가장 성능 좋은 분류기를 설계하시오

In [67]:
parameters = {'kernel':['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [ 0.1,0.01,0.001, 0.0001] ,'class_weight' : ['balanced']}
svc = svm.SVC(gamma="scale")
greed = GridSearchCV(svc, parameters, cv=10)
greed.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 10, 100, 1000], 'class_weight': ['balanced'],
                         'gamma': [0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [68]:
greed.best_params_

{'C': 1, 'class_weight': 'balanced', 'gamma': 0.1, 'kernel': 'rbf'}

In [69]:
y_predict = greed.predict(X_test)
confusion_matrix(y_test, y_predict)

array([[24,  0,  1,  0,  1,  1,  2,  0,  0,  0],
       [ 1, 15,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0, 10,  2,  1,  2,  0,  1,  2,  2],
       [ 0,  0,  0, 14,  5,  0,  0,  1,  1,  8],
       [ 0,  0,  1,  1, 12,  0,  2,  0,  4,  1],
       [ 2,  3,  0,  2,  1, 15,  0,  0,  1,  0],
       [ 2,  0,  0,  1,  0,  0, 22,  0,  0,  1],
       [ 0,  0,  0,  1,  2,  0,  0, 17,  5,  1],
       [ 1,  0,  0,  1,  1,  0,  0,  1, 17,  1],
       [ 3,  0,  4,  3,  2,  0,  2,  0,  2,  8]])

In [70]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.71      0.83      0.76        29
           1       0.83      0.94      0.88        16
           2       0.62      0.48      0.54        21
           3       0.56      0.48      0.52        29
           4       0.48      0.57      0.52        21
           5       0.83      0.62      0.71        24
           6       0.79      0.85      0.81        26
           7       0.85      0.65      0.74        26
           8       0.53      0.77      0.63        22
           9       0.36      0.33      0.35        24

    accuracy                           0.65       238
   macro avg       0.66      0.65      0.65       238
weighted avg       0.66      0.65      0.64       238



## [수정O] 테스트 데이터 준비 
- scale 함수로 정규화!

In [0]:
# Load datasets
# DataFrame 을 이용하면 편리하다.
test = pd.read_csv('/content/input2/data_test.csv')

testdata = test [['tempo', 'beats', 'chroma_stft', 'rmse',
       'spectral_centroid', 'spectral_bandwidth', 'rolloff',
       'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5',
       'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12',
       'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19',
       'mfcc20']]

testdata = testdata.values
scaler.fit(testdata)
testdata = scaler.transform(testdata)

## [수정O] 학습한 모델을 이용하여 테스트 데이터의 라벨 구하기


In [72]:
result = greed.predict(testdata)
result = result.reshape(-1,1)
print(result.shape)

(50, 1)


## [수정O] 결과 파일로 저장하여 Kaggle 제출 준비
- 코드 변경 가능하나 기본 코드 사용을 권장함 

In [73]:
# numpy 를 Pandas 이용하여 결과 파일로 저장

import pandas as pd

print(result.shape)
df = pd.DataFrame(result, columns=["label"])
df.index = np.arange(1,len(df)+1)
df.index.name = 'id'
df.to_csv('yhchoi-p5.csv',index=True, header=True)

(50, 1)


## [수정O] Kaggle 에 result_yk.csv 파일 제출
- 제출 파일명 본인에게 맞게 수정
- "ykchoi-1810"" 본인의 이름과 제출 시간으로 수정

In [74]:
! kaggle competitions submit -c 2019-pr-midterm-musicclassification -f yhchoi-p5.csv -m "yhchoi-191031"

100% 250/250 [00:02<00:00, 117B/s]
Successfully submitted to 2019-PR-Midterm-MusicClassification